In [ ]:
from python_scripts.connector import *
from python_scripts.helper import *

from eval_scripts.evalscript_true_color import *
from eval_scripts.evalscript_ndvi import *
from eval_scripts.evalscript_ndvi_values  import *

from sentinelhub import MimeType, CRS, BBox, SentinelHubRequest, SentinelHubDownloadClient,DataCollection, bbox_to_dimensions, SHConfig
from sentinelhub.geo_utils import to_wgs84
from sentinelhub.constants import CRS
from sentinelhub import SentinelHubCatalog

import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


In [ ]:
def get_time_slots(start, end, chunks):
    """
    Get the avalable time windows (time slots)
    """ 
    datetime_start = datetime.datetime.strptime(start, '%Y-%m-%d')
    datetime_end   = datetime.datetime.strptime(end, '%Y-%m-%d')
    
    tdelta = (datetime_end - datetime_start) / chunks
    edges = [(datetime_start + i*tdelta).date().isoformat() for i in range(chunks)]
    slots = [(edges[i], edges[i+1]) for i in range(len(edges)-1)]
    return slots
    

In [ ]:
def get_true_color_request(time_interval):
    return SentinelHubRequest(
        evalscript=evalscript_true_color,
        input_data=[
            SentinelHubRequest.input_data(
                data_collection=DataCollection.SENTINEL2_L2A,
                time_interval=time_interval,
                mosaicking_order='leastCC' # TODO: change to set coud coverage to desired percentage
            )
        ],
        responses=[
            SentinelHubRequest.output_response('default', MimeType.PNG)
        ],
        bbox=betsiboka_bbox,
        size=betsiboka_size,
        config=config
    )

In [ ]:
def bbox_converter(lng, lat):
    
    lng, lat = to_wgs84(lng, lat, CRS.WGS84)
    coords = [lng - 0.1, lat - 0.1, lng + 0.1, lat + 0.1]
    return coords

In [ ]:
def get_mask(eval_script, coordinations_file, slots, config):
    """
    Get mask from sentinel hub of the all the farmers location  and different timestamps.
    """
    no_clouds = { 
    "dataFilter": { 
        "maxCloudCoverage": 0
        } 
    } 
    # convert coordinations_file (csv file) into a dataframe
    coordinations_df = pd.read_csv(coordinations_file, sep= ";")
    
    # create a list of requests
    for index, rows in coordinations_df.iterrows(): # iterate through each location 
        
        coords = bbox_converter(rows["Longitude"], rows['Latitude'])
        
        list_of_requests = [sentinel_request(eval_script, slot, config, False, other_args=no_clouds) for slot in slots]
        list_of_request = [request.download_list[0] for request in list_of_requests]
    
        # download data with multiple threads
        data = SentinelHubDownloadClient(config=config).download(list_of_requests, max_threads=5)
    
    return data

In [ ]:
def plot_images(data, slots):
    # some stuff for pretty plots
    ncols = 2
    nrows = 2
    aspect_ratio = betsiboka_size[0] / betsiboka_size[1]
    subplot_kw = {'xticks': [], 'yticks': [], 'frame_on': False}

    fig, axs = plt.subplots(ncols=ncols, nrows=nrows, figsize=(5 * ncols * aspect_ratio, 5 * nrows),
                            subplot_kw=subplot_kw)

    for idx, image in enumerate(data):
        ax = axs[idx // ncols][idx % ncols]
        ax.imshow(np.clip(image * 2.5/255, 0, 1))
        ax.set_title(f'{slots[idx][0]}  -  {slots[idx][1]}', fontsize=10)

    plt.tight_layout()

In [ ]:
slots = get_time_slots('2017-01-01', '2022-01-01', 5)
print("Yearly time windows")
for slot in slots:
    print(slot)